In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import pandas as pd
import datetime
import sys
import re
import gc
import glob

import os
HOME = os.path.expanduser('~')
sys.path.append(f"{HOME}/kaggle/data_analysis/library/")
import utils
from utils import logger_func
logger = logger_func()
pd.set_option('max_columns', 200)
pd.set_option('max_rows', 200)
from sklearn.metrics import mean_squared_error

path_list = glob.glob('../stack/*.gz')
import pickle
import datetime
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']

In [ ]:
from tqdm import tqdm
col_term = 'hist_regist_term'
base = utils.read_df_pkl('../input/base_term*')[[key, target, col_term]]
# Indexをそろえる
base_model_path = '../stack/0207_200_stack_lgb_lr0.01_235feats_10seed_70leaves_iter1870_OUT0_CV1-5500865669760113_LB.gz'
best_model = utils.read_pkl_gzip(base_model_path)
base_idx = train[[key, col_term]]
best_model = base_idx.merge(best_model, how='inner', on=key)

best_model[col_term] = best_model[col_term].map(lambda x: 
                                          6 if 6<=x and x<=8  else 
                                          9 if 9<=x and x<=12
                                          else x
                                         )
base[col_term] = base[col_term].map(lambda x: 
                                          6 if 6<=x and x<=8  else 
                                          9 if 9<=x and x<=12
                                          else x
                                         )
train = base[~base[target].isnull()]

In [ ]:
score_list = []
ratio_list = []
blend_path_list = glob.glob('../stack/*.gz')

base = base[base[target]>-30]
best_model = best_model[best_model[target]>-30]
y_train = base[~base[target].isnull()][target].values
best_pred = best_model['pred_mean'].values[:len(y_train)]
best_score = np.sqrt(mean_squared_error(y_train, best_pred))
print(f"Best Score: {best_score}")
sys.exit()

for i, path in enumerate(blend_path_list):
    tmp = utils.read_pkl_gzip(path)
    tmp = base_idx.merge(tmp, how='inner', on=key)
    
    if 'pred_mean' in tmp.columns:
        tmp_pred = tmp['pred_mean'].values
    else:
        tmp_pred = tmp['prediction'].values
    del tmp
    gc.collect()
    
    tmp_pred = tmp_pred[:len(train)]
    tmp_score_list = []
    min_score = 100
    for ratio in np.arange(0.1, 1.0, 0.1):
        stack_pred = best_pred * ratio + tmp_pred * (1-ratio)
        y_pred = stack_pred[:len(y_train)]
    
        score = np.sqrt(mean_squared_error(y_train, y_pred))
        if score<min_score:
            min_score = score
            min_ratio = ratio
            
    score_list.append(min_score)
    ratio_list.append(min_ratio)
    
    if i%10==0:
#         print(f"BEST: {np.min(score_list)} | BEST RATIO: {ratio_list[np.argmin(score_list)]} | WORST: {np.max(score_list)} | WORST RATIO: {ratio_list[np.argmax(score_list)]}")
        print(f"BEST: {np.min(score_list)} | BEST RATIO: {ratio_list[np.argmin(score_list)]} | PATH: {blend_path_list[np.argmin(score_list)]}")
    
print("Complete!!")

In [115]:
train[col_term].value_counts().sum()

201917

In [128]:
outlier_thres = -3
term4  = train[train[col_term] == 4]
term5  = train[train[col_term] == 5]
term6  = train[train[col_term] == 6]
term9  = train[train[col_term] == 9]
term15  = train[train[col_term] == 15]
term18  = train[train[col_term] == 18]
term24  = train[train[col_term] == 24]

df_list = [
term4    
,term5 
,term6    
,term9    
,term15   
,term18   
,term24   
]


trn_idx_list = []
val_idx_list = []
train_dict = {}
valid_dict = {}
for df in df_list:
    plus  = df[df[target] >= 0]
    tmp_minus = df[df[target] <  0]
    minus = tmp_minus[tmp_minus[target] >  -30]
    out = tmp_minus[tmp_minus[target] <  -30]

    plus['outliers'] = plus[target].map(lambda x: 1 if x>=outlier_thres*-1 else 0)
    minus['outliers'] = minus[target].map(lambda x: 1 if x<=outlier_thres else 0)
    out['outliers'] = out[target].map(lambda x: 1 if x<=outlier_thres else 0)

    folds = StratifiedKFold(n_splits=fold, shuffle=True, random_state=seed)
    kfold_plus = folds.split(plus, plus['outliers'].values)
    kfold_minus = folds.split(minus, minus['outliers'].values)
    kfold_out = folds.split(out, out['outliers'].values)
    
    for i, ((p_trn_idx, p_val_idx), (m_trn_idx, m_val_idx), (o_trn_idx, o_val_idx)) in enumerate(zip(kfold_plus, kfold_minus, kfold_out)):
    
        def get_ids(df, idx):
            ids = list(df.iloc[idx, :][key].values)
            return ids
    
        trn_ids = get_ids(plus, p_trn_idx) + get_ids(minus, m_trn_idx) + get_ids(out, o_trn_idx)
        val_ids = get_ids(plus, p_val_idx) + get_ids(minus, m_val_idx) + get_ids(out, o_val_idx)
    
        # idをindexの番号にする
        trn_ids = list(train[train[key].isin(trn_ids)].index)
        val_ids = list(train[train[key].isin(val_ids)].index)
    
#         trn_idx_list.append(trn_ids)
#         val_idx_list.append(val_ids)
        if i not in train_dict:
            train_dict[i] = trn_ids
            valid_dict[i] = val_ids
        else:
            train_dict[i] += trn_ids
            valid_dict[i] += val_ids
    print(len(train_dict[i]))
# kfold = zip([trn_idx_list], [val_idx_list])
kfold = list(zip(train_dict.values(), valid_dict.values()))

27823
41947
77568
119280
133585
149928
168280


In [130]:
utils.read_pkl_gzip( '../stack/0208_071_stack_lgb_lr0.01_235feats_multi10_valterm_1seed_71leaves_iter1097_TERM18_CV3-4609004004622372_LB.gz').shape

(325540, 3)

In [ ]:
submit = pd.read_csv('../input/sample_submission.csv')
submit['target'] = blending

clf = utils.read_pkl_gzip('../stack/0112_155_outlier_classify_9seed_lgb_binary_CV0-9047260065151934_200features.gz')
clf = clf.iloc[-len(submit):, ].reset_index(drop=True)
submit.loc[clf.prediction>0.45, 'target'] = -33.1


submit.to_csv(f'../submit/{start_time[4:12]}_elo_{len(blend_list)}blender_outlier_clf0.45_postprocessing.csv', index=False)